# Energy Aspects wick_airport_wick_airport_data Analyst case study
You are a data analyst at Energy Aspects. <br>
The gas team needs a model to help them understand gas demand in Northern Scotland. <br>
They believe Gas demand is tied up to temperatures and tasked you to forecast minimum temperatures in the next 24 months.<br>
For this exercise, you will use weather data from Wick Airport as a proxy for Northern Scotland weather.<br>
The language to be used is Python.<br>
You can use any library you may find useful for the task.<br>

### Case study assessment
- Do not overcomplexify your answers. We value our team's ability to give simple answers to simple questions
- Focus on clean code and short, high level explainations of choices
- Do not spend time tuning models to get accurate results. We will not assess you on model accuracy / tuning
- We will assess relevance of analytics, visualisations, model choice and reasonings

## Data source and description

wick_airport_weather_data.csv<br>
https://www.metoffice.gov.uk/research/climate/maps-and-data/historic-station-data<br>

Wick Airport<br>
Location  336500E 952200N, Lat 58.454 Lon -3.088, 36m amsl<br>
Estimated data is marked with a * after the value.<br>
Missing data (more than 2 days missing in month) is marked by  ---.<br>
Sunshine data taken from an automatic Kipp & Zonen sensor marked with a #, otherwise sunshine data taken from a Campbell Stokes recorder.<br>
- Mean daily maximum temperature (tmax): degC
- Mean daily minimum temperature (tmin): degC
- Days of air frost (af): days
- Total rainfall (rain): mm
- Total sunshine duration (sun): hours

# Import data
Import data from data/wick_airport_weather_data.csv

In [60]:
import pandas as pd
import numpy as np

wick_airport_data_file_path = "data/wick_airport_weather_data.csv"
wick_airport_data = pd.read_csv(wick_airport_data_file_path)

# Extract and transform
Make all the cleaning and transformation steps you judge necessary to analyse the data

In [66]:
def generate_data_cleanliness_stats(df):
    # Check for missing values and provide a summary of the dataset
    print(df.info(), df.isna().sum())


def clean_data(df):
    def process_estimated_data(df, columns_to_check):
        """
        Processes columns in a df to handle estimated data marked with '*'.
        
        Args:
        df (pd.DataFrame): df with columns that may contain estimated data values marked with '*'.
        columns_to_check (list)
        """
        
        for col in columns_to_check:            
            # Remove the asterisk and convert column to numeric type
            if df[col].dtype == object:
                df[col] = pd.to_numeric(df[col].str.replace('*', ''), errors='coerce')

        return df
    
    def interpolate_column(df, column, method):
        if df[column].isnull().any():
            df[column] = df[column].interpolate(method)

        return df

    # Drop columns with excessive missing data or that are not needed
    columns_to_drop = ['sun', 'status']
    df.drop(columns=[col for col in columns_to_drop if col in df.columns], inplace=True)

    # Columns that may contain estimated values
    columns_to_check = ['tmax', 'tmin', 'rain']
    df = process_estimated_data(df, columns_to_check)

    # Replace placeholders like '---' with NaN and convert temperature readings from strings to floats, if not already done
    if df['tmax'].dtype == object:
        columns_to_convert = ['tmax', 'tmin', 'af', 'rain']
        df[columns_to_convert] = df[columns_to_convert].replace('---', np.nan).apply(pd.to_numeric, errors='coerce')

    # Convert the 'yyyy' and 'mm' columns into a single datetime column representing the first day of each month
    if "date" not in df.columns:
        df['date'] = pd.to_datetime(df['yyyy'].astype(str) + '-' + df['mm'].astype(str) + '-01')
        df.set_index('date', inplace=True)

    # Fill missing temperature values using interpolation
    df = interpolate_column(df, "tmin", "time")
    df = interpolate_column(df, "tmax", "time")
        
    return df


def check_data_integrity(df):
    """
    Checks the integrity of weather data for potentially erroneous entries.
    This function prints out rows that contain values outside typical ranges for temperature and rainfall.
    
    Args:
    df: df containing weather data with columns for 'tmax', 'tmin', and 'rain'.
    """
    # Define reasonable ranges for temperature and rainfall
    temp_range = (-30, 35)
    rain_range = (0, 1000)

    # Identify outliers
    temp_outliers = df[~df['tmax'].between(*temp_range) | ~df['tmin'].between(*temp_range)]
    rain_outliers = df[~df['rain'].between(*rain_range)]
    
    # Print potentially erroneous data
    if not temp_outliers.empty:
        print("Potential temperature errors found:")
        print(temp_outliers[['tmax', 'tmin']])
    
    if not rain_outliers.empty:
        print("Potential rainfall errors found:")
        print(rain_outliers['rain'])

generate_data_cleanliness_stats(wick_airport_data)
wick_airport_data = clean_data(wick_airport_data)
generate_data_cleanliness_stats(wick_airport_data)

check_data_integrity(wick_airport_data)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1302 entries, 1914-01-01 to 2022-06-01
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   yyyy    1302 non-null   int64  
 1   mm      1302 non-null   int64  
 2   tmax    1302 non-null   float64
 3   tmin    1302 non-null   float64
 4   af      1302 non-null   object 
 5   rain    1302 non-null   float64
dtypes: float64(3), int64(2), object(1)
memory usage: 71.2+ KB
None yyyy    0
mm      0
tmax    0
tmin    0
af      0
rain    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1302 entries, 1914-01-01 to 2022-06-01
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   yyyy    1302 non-null   int64  
 1   mm      1302 non-null   int64  
 2   tmax    1302 non-null   float64
 3   tmin    1302 non-null   float64
 4   af      1302 non-null   object 
 5   rain    1302 non-null   float64
dtypes: float64(3),

# Data exploration and charting
Perform Exploratory Data Analysis on the dataset.<br>
Present your findings in a few lines and charts.

In [67]:
wick_airport_data.head()

,yyyy,mm,tmax,tmin,af,rain
date,,,,,,
1914-01-01,1914,1,5.9,0.9,---,48.3
1914-02-01,1914,2,7.7,2.3,---,66.0
1914-03-01,1914,3,6.4,1.1,---,76.7
1914-04-01,1914,4,10.8,4.4,---,27.9
1914-05-01,1914,5,10.7,4.4,---,61.0


# Modelling
Choose a model, backtest it and generate forecasts for the 24 months ahead.<br>
Explain your model choice in a few lines and present backtesting / forecasting results using tables and charts understandable for analysts who do not write code or build statistical models.
